# Water molecule

This notebook demonstrates how to combine `pyscf` with the DMRG implementation in `chemtensor` for approximating the ground state of the water molecule.

In [1]:
import numpy as np
import chemtensor

# PySCF (https://pyscf.org/) defines a molecular orbital basis, computes overlap integrals and runs other computational methods for comparison.
import pyscf

### Define the molecule and perform reference calculations

In [2]:
h2o_atoms = [
    ["O", ( 0.,   0.,   0.)],
    ["H", ( 0.75, 0.47, 0.)],
    ["H", (-0.75, 0.47, 0.)],
]
mol = pyscf.M(atom=h2o_atoms, basis="sto-3g")

In [3]:
# run Hartree-Fock
hf = mol.HF().run()

converged SCF energy = -74.9307084820999


In [4]:
# run coupled-cluster with single and double excitations (CCSD), for comparison
ccsd = pyscf.cc.CCSD(hf).run()
ccsd.e_tot

E(CCSD) = -74.97016403895397  E_corr = -0.0394555568540221


<class 'pyscf.cc.ccsd.CCSD'> does not have attributes  converged


-74.97016403895397

In [5]:
# run full configuration interaction (FCI)
fcisolver = pyscf.fci.FCI(hf)
en_fci, _ = fcisolver.kernel()
en_fci

-74.97027268959359

### Electron overlap integrals

In [6]:
# overlap integrals in atomic basis
h1_ao  = mol.intor("int1e_kin") + mol.intor("int1e_nuc")
eri_ao = mol.intor("int2e")

In [7]:
# transform to molecular orbital basis
h1_mo = np.einsum("pi,pq,qj->ij", hf.mo_coeff, h1_ao, hf.mo_coeff)
eri_mo = pyscf.ao2mo.kernel(eri_ao, hf.mo_coeff)
print(h1_mo.shape)
print(eri_mo.shape)

(7, 7)
(7, 7, 7, 7)


In [8]:
# convert to physicists' convention
tkin = h1_mo
vint = np.transpose(eri_mo, (0, 2, 1, 3))

### Construct Hamiltonian as MPO and run two-site DMRG

In [9]:
hamiltonian = chemtensor.construct_spin_molecular_hamiltonian_mpo(tkin, vint)
# virtual bond dimensions
hamiltonian.bond_dims

[1, 36, 58, 96, 96, 58, 36, 1]

In [10]:
# local physical quantum numbers (number of electrons and spin)
[chemtensor.decode_quantum_number_pair(qnum) for qnum in hamiltonian.qsite]

[(0, 0), (1, -1), (1, 1), (2, 0)]

In [11]:
# overall quantum number sector of quantum state (number of electrons and spin)
q_pnum = 10
q_spin = 0
qnum_sector = chemtensor.encode_quantum_number_pair(q_pnum, q_spin)

In [12]:
# run two-site DMRG
psi, en_sweeps, entropy = chemtensor.dmrg(hamiltonian, num_sweeps=6, maxiter_lanczos=25, tol_split=1e-9, qnum_sector=qnum_sector)

### Evaluate results

In [13]:
# virtual bond dimensions of optimized MPS
psi.bond_dims

[1, 4, 13, 27, 19, 16, 4, 1]

In [14]:
# energy after each DMRG sweep
en_sweeps

[-84.8890344353728,
 -84.88903443537976,
 -84.8890344353798,
 -84.88903443537997,
 -84.8890344353799,
 -84.88903443537991]

In [15]:
# add nuclear repulsion energy
en_dmrg = en_sweeps[-1] + hf.energy_nuc()
en_dmrg

-74.97027263503894

In [16]:
# difference to CCSD energy
ccsd.e_tot - en_dmrg

0.00010859608497071349

In [17]:
# difference to FCI energy
en_dmrg - en_fci

5.455464702208701e-08